# Coding practice \#5: Due end of class, December 10

Files needed = ('JEC.xls', 'JEC_data_description.pdf', 'SP500_daily.csv' )

Answer the questions below in a jupyter notebook. You can simply add cells to this notebook and enter your answers. When you are finished, print the notebook and hand it in during class. \[To print: From the file menu, choose 'print preview' which will open a new tab with the notebook ready to print. Please print on both sides of paper if possible.\]

A reminder: Ruhl's office hours are T/R 2:30PM-3:30PM in Soc Sci 7444 and McWeeny's office hours are Monday 9:30AM-11:30AM in Soc Sci 6470. 

*You should feel free to discuss the coding practice with your classmates, but the work you turn in should be your own.*


# Exercise 0: Your name
Replace 'Your name' above with your actual name. Enter it as last name, first name. Do not forget the comma!

# Exercise 1: Estimating a demand curve

This problem is taken from empirical exercise 12.1 in the textbook *Introduction to Econometrics*, 10th edition, by James H. Stock and Mark W. Watson.

During the 1880s, a cartel known as the Joint Executive Committee (JEC) controlled the rail transport of grain from the Midwest to eastern cities in the United States. The cartel preceded the Sherman Antitrust Act of 1890, and it legally operated to increase the price of grain above what would have been the competitive price. From time to time, cheating by members of the cartel brought about a temporary collapse of the collusive price-setting agreement.

In this exercise, you will use variations in supply associated with the cartel's collapses to estimate the elasticity of demand for rail transport of grain.

The file 'JEC.xls' contains the data required for this exercise. It contains weekly observations on the rail shipping price and other factors from 1880 to 1886. See 'JEC_data_description.pdf' for a detailed description of the variables.

Suppose that the **demand curve equation** for rail transport of grain is specified as 

$$\ln(Q_i) = \beta_0 + \beta_1 \ln(P_i) + \beta_2 Ice_i + \sum_{j=1}^{12} \beta_{2+j} Seas_{j,i} + u_i,$$

where $Q_i$ is the total tonnage of grain shipped in week $i$, $P_i$ is the price of shipping one ton (2,000 pounds) of grain by rail, $Ice_i$ is a binary variable that is equal to 1 if the Great Lakes are not navigable because of ice, and $Seas_{j,i}$ is a binary variable that captures seasonal variation in demand. $Ice$ is included because grain could also be transported by ship when the Great Lakes were navigable.

### Part (a)

Load the data as a Pandas dataframe. Then estimate the demand curve equation by OLS using the HC3 estimator for the standard errors. Print the summary table of the results. What is the estimated value of the price elasticity of demand and its standard error? Print these values after the results table, rounding to 3 decimal places.

### Part (b)

Explain why the interaction of supply and demand could make the OLS estimator of the elasticity biased.

### Part (c)

Consider using the variable $cartel_i$ as an instrumental variable for $\ln(P_i)$ (see JEC_data_description.pdf for this variable's definition). Use economic reasoning to argue whether $cartel_i$ plausibly satisfies the two conditions for a valid instrument.

### Part (d)

Estimate the demand equation using two stage least squares with $cartel_i$ as an instrument for $\ln(P_i)$. Print the results summary table. What is the estimated value of the price elasticity of demand and its standard error? Print these values after the results table, rounding to 3 decimal places.

**NOTE:** You might need to install the `linearmodels` package if you didn't attend the class on IV regression. To do so, open a command window and type

`pip install --user linearmodels`

then hit enter.

# Exercise 2: Forecasting U.S. GDP growth

In this exercise, we will forecast the growth rate of U.S. GDP for the next 9 quarters: the fourth quarter of 2018 through the fourth quarter of 2020. Some economists believe that the U.S. economy will enter a recession by the end of 2020:
* According to a survey conducted by the National Association for Business Economists, two-thirds of the economists surveyed believed the U.S. will enter a recession by the end of 2020. [(Source)](https://apnews.com/5df2300faa8a4f74b9b0f3ae2b6c64aa)
* In June, former Federal Reserve Chairman Ben Bernanke said "\[The stimulus\] is going to hit the economy in a big way this year and next year, and then in 2020 Wile E. Coyote is going to go off the cliff." [(Source)](https://www.bloomberg.com/news/articles/2018-06-07/bernanke-says-u-s-economy-faces-wile-e-coyote-moment-in-2020)

What do you think? 

We'll expand on the model used in class: instead of using a pure autoregressive model, we will add other indicators that are predictive of GDP growth to the model. Follow the instructions below to create the forecast.

### Part (a): Importing and preparing the data

1. Use `pandas_datareader` and the FRED API to import the following variables into a Pandas dataframe. **Set the start date as January 1, 1980.**

    * Real GDP growth rate (quarterly): code 'A191RL1Q225SBEA'
    * Unemployment rate (monthly): code 'UNRATE'
    * Consumer price index (monthly): code 'CPIAUCSL'
    * New housing starts (monthly): code 'HOUST'
    
Once the data are imported: 

* Give these variables more reasonable names. 
* Resample the data to the quarterly frequency. 
* Create a variable containing the inflation rate, calculated as the percentage change in the consumer price index. 
* Print the last 5 rows of the resulting dataframe.

2.  Import the file 'SP500_daily.csv'  to obtain daily closing values of the S&P 500 Index. (Strangely, all of the finance APIs compatible with pandas_datareader only import data since 2010 or so, so we'll use a CSV file for this instead.) 


* Set the dates as the index. 
* Resample to the quarterly level. 
* Create a new column containing the quarterly growth rate of the S&P 500. 
* Print the last 5 rows of the resulting data frame.

3. Now merge the two data frames created above. Then, keep only the columns associated with the GDP growth rate, the unemployment rate, the inflation rate, new housing starts, and the S&P 500 growth rate. Print the last 5 rows of the data frame.

### Part (b): The model and creating lagged and forward values

Our goal is to forecast the growth rate of GDP $h$ quarters into the future, for $h=1,2,\dots,9$. Do do so we will run a separate regression for each forecast horizon, and then use the most recent values of the data (i.e., the values for the third quarter of 2018) to create the predicted value of GDP growth for each $h$.

In particular, we will use the model

$$
\begin{align}
gGDP_{t+h} = & \rho^h_0 + \rho^h_1 gGDP_t + \rho^h_2 gGDP_{t-1} + \rho^h_3 unemp_t + \rho^h_4 unemp_{t-1} \\
             & \\
             & + \rho^h_5 inflation_t + \rho^h_6 inflation_{t-1} + \rho^h_7 housing_t + \rho^h_8 housing_{t-1} \\
             & \\
             & + \rho^h_9 gSP500_t + \rho^h_{10} gSP500_{t-1} + \varepsilon_{t+h} \\
\end{align}
$$

where $gGDP_t$ is the GDP growth rate in quarter $t$, $unemp_t$ is the unemployment rate, $inflation_t$ is the inflation rate, $housing_t$ is the number of new housing starts, and $gSP500_t$ is the growth rate of the S&P 500. Note that the parameters are indexed by $h$, meaning that their values will be different for different forecast horizons.

Follow the instructions below.

1. Before estimating the model, create new columns that contain the lagged values of the explanatory variables: $gGDP_{t-1}, umep_{t-1}, inflation_{t-1}, housing_{t-1}$, and $gSP500_{t-1}$. Use the .shift() method to do so.
2. We also need to create columns that contain the **forward** values of GDP growth, as these will be our dependent variables (i.e., $gGDP_{t+h}$ for $h=1,2,\dots,9$). You can either do this manually or write a loop. Name these columns 'F1gdp', 'F2gdp', etc. Again, use the .shift() method. Print the last 5 rows of the resulting dataframe.

### Part (c): Estimating the model

Now we can estimate the model and create our forecast of GDP growth. This will require running 9 separate regressions and then obtaining the 9 predicted values. Follow the instructions below.

1. Store the values of the right hand side variables for the third quarter of 2018 (i.e., '2018-09-30') as a series called `current_values`.
2. Create an empty list called `forecast_list`. Then, append the value of the GDP growth rate for the third quarter of 2018 to this list. (We will append each value of our forecast to this list as we go along - adding the current value of GDP growth just makes plotting easier.)
3. Write a for-loop that iterates through each forecast horizon $(h=1,2,\dots,9)$ and estimates the model described in part (b) using OLS. The loop should do the following:
    1. Fit the model for the given time horizon.
    2. Print the summary table of results **for the first regression only (i.e., for $h=1$).**
    3. Use the `.predict()` method and the series `current_values` to predict the future value of GDP for the current $h$. Then append this value to `forecast_list`.
    
Print the values of `forecast_list` to see the predicted GDP growth rates.


### Part (d): Plotting the historical data and the forecast.

1. Use the following line of code to create an object containing the values of the dates corresponding to each quarter from the third quarter of 2018 to the fourth quarter of 2020. These are the 'x' data you need for plotting the forecasts:
```python
future_dates = pd.date_range('2018-09-30', periods=10, freq='Q')
```
2. Now, create a plot of the historical GDP growth rates along with the forecasted values. **For the historical data, plot only the values since December 31, 2007 ('2007-12-31').** Make the historical data a solid black line and the forecast a blue dashed line. Make other aspects of the plot look nice, including appropriate labels and a legend.

If you've followed all these directions, then your forecast should have a "U" shape: we're predicting GDP growth to slow slightly but then recover over the next few years. We do not appear to be predicting a recession, as GDP growth is not forecasted to be negative.